In [2]:
import pandas as pd
import numpy as np
import nltk
import textwrap
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df = pd.read_csv('df_total.csv')

In [5]:
df.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [6]:
df["news"][2]

'En una industria históricamente masculina como lo es la aviación Viva presentó su avión rosado A320NEO que apuesta por la equidad de género la lucha contra el cáncer de mama la inclusión y la diversidad.Desde Francia llegó Go Pink que tuvo un precio promedio de US50 millones convirtiéndose en la aeronave número 20 de las 21 con las que finalizará el año esta aerolínea.En Viva estamos trabajando muy fuerte para que haya más mujeres. Actualmente el grupo ejecutivo está compuesto por 42 mujeres pero la meta es llegar en 2022 con 50 de ellas en nuestro staff señaló Félix Antelo CEO de Viva.Asimismo agregó que están evaluando traer otro avión con esta propuesta para el próximo año así como proyectan un crecimiento de 60 en sillas para el último trimestre de 2021 en comparación con el mismo periodo de 2019.Vamos a estar moviendo más de dos millones de pasajeros para finalizar el año es un número ambicioso pero estamos contentos por la reacción del mercado y porque muchos pasajeros están eli

In [10]:
doc = df['news'].sample()

In [11]:
print(doc.iloc[0])

Más de 600 jóvenes de nueve nacionalidades diferentes han participado en la 7ª edición del Hackathon BBVA. El Reto Metaverso, en el que el banco proponía experimentar con soluciones financieras inmersivas, ha sido uno de los más concurridos, atrayendo a 82 competidores distribuidos en 16 equipos. El equipo que se ha alzado con el premio ha propuesto un juego virtual para fomentar la educación financiera entre las generaciones más jóvenes, que simula situaciones reales como reformar una casa o la compra de un coche y propone el uso de un ‘token’ en forma de  moneda virtual. Sus seis componentes, procedentes de España y Perú, se llevan un premio en metálico y la posibilidad de colaborar con BBVA en futuros proyectos de desarrollo.
La propuesta ganadora del reto del metaverso en el Hackathon BBVA 2022, BBVA Metaverse Academy, plantea una plataforma inmersiva gamificada que ayuda a los ‘centennials’ a adquirir hábitos bancarios y comprender la funcionalidad de cada producto financiero enfr

In [14]:
doc2 = textwrap.fill(doc.iloc[0], replace_whitespace=False, fix_sentence_endings=True)

In [16]:
print(doc2)

Más de 600 jóvenes de nueve nacionalidades diferentes han participado
en la 7ª edición del Hackathon BBVA. El Reto Metaverso, en el que el
banco proponía experimentar con soluciones financieras inmersivas, ha
sido uno de los más concurridos, atrayendo a 82 competidores
distribuidos en 16 equipos.  El equipo que se ha alzado con el premio
ha propuesto un juego virtual para fomentar la educación financiera
entre las generaciones más jóvenes, que simula situaciones reales como
reformar una casa o la compra de un coche y propone el uso de un
‘token’ en forma de  moneda virtual.  Sus seis componentes,
procedentes de España y Perú, se llevan un premio en metálico y la
posibilidad de colaborar con BBVA en futuros proyectos de desarrollo.
La propuesta ganadora del reto del metaverso en el Hackathon BBVA
2022, BBVA Metaverse Academy, plantea una plataforma inmersiva
gamificada que ayuda a los ‘centennials’ a adquirir hábitos bancarios
y comprender la funcionalidad de cada producto financiero
en

In [21]:
#Dividir el texto en lineas
lineas = doc2.split('. ')

In [22]:
lineas

['Más de 600 jóvenes de nueve nacionalidades diferentes han participado\nen la 7ª edición del Hackathon BBVA',
 'El Reto Metaverso, en el que el\nbanco proponía experimentar con soluciones financieras inmersivas, ha\nsido uno de los más concurridos, atrayendo a 82 competidores\ndistribuidos en 16 equipos',
 ' El equipo que se ha alzado con el premio\nha propuesto un juego virtual para fomentar la educación financiera\nentre las generaciones más jóvenes, que simula situaciones reales como\nreformar una casa o la compra de un coche y propone el uso de un\n‘token’ en forma de  moneda virtual',
 ' Sus seis componentes,\nprocedentes de España y Perú, se llevan un premio en metálico y la\nposibilidad de colaborar con BBVA en futuros proyectos de desarrollo.\nLa propuesta ganadora del reto del metaverso en el Hackathon BBVA\n2022, BBVA Metaverse Academy, plantea una plataforma inmersiva\ngamificada que ayuda a los ‘centennials’ a adquirir hábitos bancarios\ny comprender la funcionalidad de ca

In [23]:
tokenizar = TfidfVectorizer(
    stop_words=stopwords.words('spanish'),
    norm = 'l1', #Con esto estamos diciendo que el valor máximo de cada una de las líneas no puede sumar más de uno (van de 0 a 1)
)

In [24]:
#Construimos la matriz con las lineas
x = tokenizar.fit_transform(lineas)

In [26]:
x
#14 es el nro de filas, 423 es el nro de columnas
#las filas son las lineas
#las columnas son las palabras
#dentro están los scores según la cant de veces que cada palabra aparece en cada linea

<14x423 sparse matrix of type '<class 'numpy.float64'>'
	with 551 stored elements in Compressed Sparse Row format>

In [25]:
def obtener_score(tfidf_row):
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

#Obtenemos el score de la linea a partir del promedio de los scores por cada palabra
#solamente cuando no es una fila llena de 0, porque si no, va a dar un error al intentar
#dividir por 0

In [28]:
scores = np.zeros(len(lineas))
for i in range(len(lineas)):
    score = obtener_score(x[i,:])
    scores[i] = score

In [31]:
scores

array([0.1       , 0.0625    , 0.04347826, 0.01666667, 0.00934579,
       0.02857143, 0.01818182, 0.11111111, 0.02173913, 0.01587302,
       0.01923077, 0.04347826, 0.03571429, 0.04166667])

In [34]:
sort_idx = np.argsort(-scores)

In [35]:
sort_idx

array([ 7,  0,  1,  2, 11, 13, 12,  5,  8, 10,  6,  3,  9,  4],
      dtype=int64)

In [37]:
print("Resumen:")
oraciones = []
for i in range(0,5):
    oraciones.append([sort_idx[i], scores[sort_idx[i]], lineas[sort_idx[i]]])
    print(f"{scores[sort_idx[i]]}:{lineas[sort_idx[i]]}")

Resumen:
0.1111111111111111:Para ellos supone, además, una
pasarela de visibilización y la posibilidad de establecer sinergias
con BBVA en el futuro
0.09999999999999999:Más de 600 jóvenes de nueve nacionalidades diferentes han participado
en la 7ª edición del Hackathon BBVA
0.06250000000000001:El Reto Metaverso, en el que el
banco proponía experimentar con soluciones financieras inmersivas, ha
sido uno de los más concurridos, atrayendo a 82 competidores
distribuidos en 16 equipos
0.04347826086956523: El equipo que se ha alzado con el premio
ha propuesto un juego virtual para fomentar la educación financiera
entre las generaciones más jóvenes, que simula situaciones reales como
reformar una casa o la compra de un coche y propone el uso de un
‘token’ en forma de  moneda virtual
0.043478260869565216: Con esta información, combinada
con el análisis de patrones de comportamiento (por ejemplo, compras
frecuentes en farmacias o visitas a hospitales), el equipo ha
planteado un algoritmo que pr

In [39]:
#Ordenar el resumen
oraciones_ord = sorted(oraciones, key=lambda y: y[0])

for item in oraciones_ord:
    print(item[2])

Más de 600 jóvenes de nueve nacionalidades diferentes han participado
en la 7ª edición del Hackathon BBVA
El Reto Metaverso, en el que el
banco proponía experimentar con soluciones financieras inmersivas, ha
sido uno de los más concurridos, atrayendo a 82 competidores
distribuidos en 16 equipos
 El equipo que se ha alzado con el premio
ha propuesto un juego virtual para fomentar la educación financiera
entre las generaciones más jóvenes, que simula situaciones reales como
reformar una casa o la compra de un coche y propone el uso de un
‘token’ en forma de  moneda virtual
Para ellos supone, además, una
pasarela de visibilización y la posibilidad de establecer sinergias
con BBVA en el futuro
 Con esta información, combinada
con el análisis de patrones de comportamiento (por ejemplo, compras
frecuentes en farmacias o visitas a hospitales), el equipo ha
planteado un algoritmo que predice cuándo un cliente puede pertenecer
a un grupo de adultos mayores o con discapacidad
